In [ ]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH
    
#import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["AUTOGRAPH_VERBOSITY"] = "0"

In [ ]:

import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)


datasetPathGeneratedFaces =  configParser.get('evaluate_imagen', 'test_datasetPathGeneratedFaces')
generated_face_table_name =  configParser.get('evaluate_imagen', 'generated_face_table_name')
evaluation_results_folder = configParser.get('evaluate_imagen', 'evaluation_results_folder')
datasetPathFaces =  configParser.get('evaluate_imagen', 'test_datasetpathfaces')

datasetPathGeneratedFaces_pooled =  configParser.get('evaluate_imagen', 'test_datasetpathgeneratedfaces_pooled')
datasetPathFaces_pooled =  configParser.get('evaluate_imagen', 'test_datasetpathfaces_pooled')

datasetPathDatabase =  configParser.get('COMMON', 'test_datasetPathDatabase') + '/dataset.db'


cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))




In [ ]:
#TODO document jupyter
import pickle
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

from textwrap import wrap
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [ ]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [ ]:
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
audio_length_used =  configParser.get('evaluate_imagen', 'audio_length_used') 
model_filename =  configParser.get('evaluate_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('evaluate_imagen', 'openl3_mode')

In [ ]:
import lancedb
uri = datasetPathDatabase
db = lancedb.connect(uri)

table_generated = db.open_table(generated_face_table_name).to_pandas()
#table_generated = table_generated.loc[table_generated.index.repeat(5)].reset_index(drop=True)
table_generated


In [ ]:
#try:
#  db.drop_table(generated_face_table_name)
#except:
#  print("")

In [ ]:
import multiprocessing
try:
    multiprocessing.set_start_method('spawn')
except:
    print("")

In [ ]:
import pathlib
import os
from random import randrange


uri = datasetPathDatabase
db = lancedb.connect(uri)





In [ ]:
audio_folder = output_folder = r'imagen_testing_folder' 
if not os.path.exists(audio_folder):
    os.makedirs(audio_folder)

In [11]:
from testing_imagen_face import extract_face_rep,extract_face_attributes
import pathlib
import shutil


2023-10-26 23:41:34.797827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:


i = 0
for index, row in table_generated.iterrows():


    face_file = row['generated_face_path']
    user = row['user']
    id_true= int(row['id_true'])

    q = multiprocessing.Queue()
    try:
        proc = multiprocessing.Process(target=extract_face_rep, args=(q,face_file,
                                                                image_size,output_folder,))
        proc.start()
        proc.join()
    except:
        continue
    with open(output_folder + '/' + 'vgg_generated_face.pickle', 'rb') as handle:
        vgg = pickle.load(handle)
        #print("++++++" + str(vgg))

    try:

        proc = multiprocessing.Process(target=extract_face_attributes, args=(q,face_file,))
        proc.start()
        proc.join()


        gender = q.get()
        if(gender == 'Error'):
            continue
        ethnicity = q.get()
        if(ethnicity == 'Error'):
            continue
        age = q.get()
        if(age == 'Error'):
            continue

    except:
        continue



    tbl = db.open_table("video_stage1")
    result = tbl.search(vgg).limit(10).to_df()

    match_1 = result.iloc[0]['user']
    match_2 = result.iloc[1]['user']
    match_3 = result.iloc[2]['user']
    match_4 = result.iloc[3]['user']
    match_5 = result.iloc[4]['user']
    match_6 = result.iloc[5]['user']
    match_7 = result.iloc[6]['user']
    match_8 = result.iloc[7]['user']
    match_9 = result.iloc[8]['user']
    match_10= result.iloc[9]['user']
    

    df_table = pd.DataFrame()
    df_table = df_table.append({'id': i , 'id_true':id_true,
                    'generated_face_path': face_file,'user':user,'age':age,'gender':gender,'ethnicity':ethnicity,
                    'match_1':match_1,'match_2':match_2,'match_3':match_3,'match_4':match_4,'match_5':match_5,
                    'match_6':match_6,'match_7':match_7,'match_8':match_8,'match_9':match_9,'match_10':match_10,
        'vector' : vgg,'stage': 1}, ignore_index=True)
    
    if(i == 0):
        try:
            db.create_table(generated_face_table_name + str(2), df_table)
        except:
            db.drop_table(generated_face_table_name + str(2))
            db.create_table(generated_face_table_name + str(2), df_table)
    else:
        tbl = db.open_table(generated_face_table_name + str(2))
        tbl.add(df_table)


    print("---------------" + str(i) + "----------------")
    i = i + 1


shutil.rmtree(output_folder)


1
2
3
4
5
6
7
8
9


Action: race: 100%|██████████| 3/3 [00:01<00:00,  2.25it/s]  


10
11
12
13
14
---------------0----------------
a
1
2
3
4
5
6
7
8
9


Action: race: 100%|██████████| 3/3 [00:01<00:00,  2.05it/s]  


10
11
12
13
14
---------------1----------------
a
1
2
3
4
5
6
7
8
9


...

10
11
12
13
14
---------------9----------------
a
1
2
3
4
5
6
7
8
9


Action: race: 100%|██████████| 3/3 [00:01<00:00,  2.29it/s]  


10
11
12
13
14
---------------10----------------
a
1
2
3
4
5
6
7
8
9


Action: race: 100%|██████████| 3/3 [00:01<00:00,  2.41it/s]  


10
11
12
13
14
---------------11----------------
a


In [24]:
tbl = db.open_table(generated_face_table_name + str(2))

In [25]:
ptable = tbl.to_pandas()
ptable

,id,id_true,generated_face_path,user,age,gender,ethnicity,match_1,match_2,match_3,match_4,match_5,match_6,match_7,match_8,match_9,match_10,vector,stage
0,0,0,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00001,32,Man,asian,id00003,id00001,id00012,id00007,id00008,id00006,id00004,id00002,id00004,id00010,"[0.012904658, 0.007864798, 0.015817596, 0.0268...",1
1,1,1,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00002,32,Man,white,id00001,id00003,id00012,id00007,id00008,id00004,id00004,id00009,id00006,id00002,"[0.009156784, 0.006567205, 0.013632283, 0.0206...",1
2,2,2,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00003,31,Man,white,id00001,id00003,id00012,id00008,id00007,id00006,id00004,id00004,id00002,id00009,"[0.010348763, 0.0059919124, 0.015155687, 0.025...",1
3,3,3,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00004,29,Man,white,id00003,id00001,id00006,id00008,id00012,id00007,id00002,id00010,id00004,id00004,"[0.014950133, 0.007072129, 0.020863913, 0.0317...",1
4,4,4,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00004,32,Man,asian,id00001,id00003,id00008,id00012,id00007,id00002,id00004,id00006,id00004,id00009,"[0.01290362, 0.0041051637, 0.012694264, 0.0234...",1
5,5,5,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00005,33,Man,white,id00001,id00003,id00012,id00007,id00008,id00004,id00006,id00004,id00002,id00009,"[0.011732841, 0.0035378898, 0.015527209, 0.026...",1
6,6,6,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00006,30,Man,white,id00003,id00001,id00008,id00007,id00006,id00002,id00004,id00012,id00010,id00004,"[0.015935998, 0.0010484763, 0.013722688, 0.024...",1
7,7,7,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00007,29,Man,asian,id00003,id00001,id00007,id00012,id00008,id00006,id00004,id00004,id00002,id00010,"[0.012244945, 0.0017155748, 0.01353641, 0.0251...",1
8,8,8,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00008,31,Man,asian,id00003,id00001,id00012,id00007,id00008,id00006,id00004,id00004,id00002,id00009,"[0.0152224125, 0.0037995481, 0.015252546, 0.02...",1
9,9,9,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00009,33,Man,white,id00003,id00001,id00012,id00008,id00006,id00007,id00002,id00004,id00004,id00009,"[0.010283488, 0.008214014, 0.020348, 0.0286479...",1


In [27]:
ptable_unique_users = ptable.drop_duplicates(subset=['user'], keep='first')
ptable_unique_users

,id,id_true,generated_face_path,user,age,gender,ethnicity,match_1,match_2,match_3,match_4,match_5,match_6,match_7,match_8,match_9,match_10,vector,stage
0,0,0,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00001,32,Man,asian,id00003,id00001,id00012,id00007,id00008,id00006,id00004,id00002,id00004,id00010,"[0.012904658, 0.007864798, 0.015817596, 0.0268...",1
1,1,1,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00002,32,Man,white,id00001,id00003,id00012,id00007,id00008,id00004,id00004,id00009,id00006,id00002,"[0.009156784, 0.006567205, 0.013632283, 0.0206...",1
2,2,2,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00003,31,Man,white,id00001,id00003,id00012,id00008,id00007,id00006,id00004,id00004,id00002,id00009,"[0.010348763, 0.0059919124, 0.015155687, 0.025...",1
3,3,3,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00004,29,Man,white,id00003,id00001,id00006,id00008,id00012,id00007,id00002,id00010,id00004,id00004,"[0.014950133, 0.007072129, 0.020863913, 0.0317...",1
5,5,5,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00005,33,Man,white,id00001,id00003,id00012,id00007,id00008,id00004,id00006,id00004,id00002,id00009,"[0.011732841, 0.0035378898, 0.015527209, 0.026...",1
6,6,6,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00006,30,Man,white,id00003,id00001,id00008,id00007,id00006,id00002,id00004,id00012,id00010,id00004,"[0.015935998, 0.0010484763, 0.013722688, 0.024...",1
7,7,7,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00007,29,Man,asian,id00003,id00001,id00007,id00012,id00008,id00006,id00004,id00004,id00002,id00010,"[0.012244945, 0.0017155748, 0.01353641, 0.0251...",1
8,8,8,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00008,31,Man,asian,id00003,id00001,id00012,id00007,id00008,id00006,id00004,id00004,id00002,id00009,"[0.0152224125, 0.0037995481, 0.015252546, 0.02...",1
9,9,9,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00009,33,Man,white,id00003,id00001,id00012,id00008,id00006,id00007,id00002,id00004,id00004,id00009,"[0.010283488, 0.008214014, 0.020348, 0.0286479...",1
10,10,10,/home/gamal/Datasets/Dataset1Test/GeneratedFac...,id00010,33,Man,white,id00003,id00001,id00006,id00012,id00008,id00007,id00002,id00004,id00010,id00004,"[0.013475849, 0.006137452, 0.017725043, 0.0321...",1


In [28]:
f= open(evaluation_results_folder + "/results.txt","w+")
f.write("No. of generated faces: " + str(len(ptable)) + "\n")
f.write("No. of unique users: " + str(len(ptable_unique_users)) + "\n")


24

In [29]:
acc_10 = int((len(tbl.search() \
   .where(""" user = match_1 or user = match_2 or user = match_3 
          or user = match_4 or user = match_5 or user = match_6 or user = match_7
          or user = match_8 or user = match_9 or user = match_10
    """).to_df()) / float(len(ptable))) * 100)
f.write("Face Recognition Accuracy within first 10 matches: " + str(acc_10) + "%\n")


55

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
display_labels = 
cm = confusion_matrix(pandas_users.gender_true, pandas_users.gender_generated,labels=display_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=display_labels,)
disp.plot(cmap="Blues")
disp.figure_.savefig(evaluation_results_folder + "/gender_cm.png",dpi=300)

In [18]:
acc_5 = int((len(tbl.search() \
   .where(""" user = match_1 or user = match_2 or user = match_3
          or user = match_4 or user = match_5 
    """).to_df()) / float(len(ptable))) * 100)
f.write("Face Recognition Accuracy within first  5 matches: " + str(acc_5) + "%\n")

55

In [19]:
acc_3 = int((len(tbl.search() \
   .where(""" user = match_1 or user = match_2 or user = match_3
    """).to_df()) / float(len(ptable))) * 100)
f.write("Face Recognition Accuracy within first  3 matches: " + str(acc_3) + "%\n")

55

In [20]:
acc_1 = int((len(tbl.search() \
   .where(""" user = match_1
    """).to_df()) / float(len(ptable))) * 100)
f.write("Face Recognition Accuracy within first  1 matches: " + str(acc_1) + "%\n")

54

In [21]:
tbl_generated = db.open_table(generated_face_table_name)
tbl_truth = db.open_table("video_stage1")

In [22]:
pandas_generated = tbl_generated.to_pandas()
pandas_users = pandas_generated[['id_true','user']]
#pandas_users = pandas_users.drop_duplicates(keep='first', inplace=False)
pandas_users["age_true"] = np.nan
pandas_users["age_generated"] = np.nan
pandas_users["gender_true"] = np.nan
pandas_users["gender_generated"] = np.nan
pandas_users["ethnicity_true"] = np.nan
pandas_users["ethnicity_generated"] = np.nan
pandas_users

,id_true,user,age_true,age_generated,gender_true,gender_generated,ethnicity_true,ethnicity_generated
0,0,id00001,NaN,NaN,NaN,NaN,NaN,NaN
1,1,id00002,NaN,NaN,NaN,NaN,NaN,NaN
2,2,id00003,NaN,NaN,NaN,NaN,NaN,NaN
3,3,id00004,NaN,NaN,NaN,NaN,NaN,NaN
4,4,id00004,NaN,NaN,NaN,NaN,NaN,NaN
5,5,id00005,NaN,NaN,NaN,NaN,NaN,NaN
6,6,id00006,NaN,NaN,NaN,NaN,NaN,NaN
7,7,id00007,NaN,NaN,NaN,NaN,NaN,NaN
8,8,id00008,NaN,NaN,NaN,NaN,NaN,NaN
9,9,id00009,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
def make_square(im, min_size=image_size, fill_color=(0, 0, 0, 0)):
    x, y = im.size
    size = max(min_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    im.close()
    return new_im

def real_image_preprocess(path):

    image = Image.open(path).convert('RGB')

    w_s = image_size / (1+2 * 0.5)
    h_s = image_size / (1+2 * 0.4)

    image = image.crop((0.4*w_s, 0.0*h_s, 1.6*w_s, 1.4*h_s))

    image = make_square(image)

    image = image.resize((begin_with_image_size,begin_with_image_size))
    im = image
    

    #print('saving')
    #image.save('opop.png')

    #print(np.array(image,np.float32).shape)

    pix = np.array(image, np.float32)
    pix = np.moveaxis(pix, -1, 0)

    pix = pix / 255
    image.close()
    im.close()
    return pix.tolist()
return image_guide

SyntaxError: 'return' outside function (3390396233.py, line 36)

In [ ]:
import cv2
from skimage.metrics import structural_similarity as calc_ssim
from skimage.metrics import mean_squared_error
psnr = []
ssim = []
mse = []
for index, row in pandas_users.iterrows():
    id_true = str(row['id_true'])
    user = row['user']
    user = "'" + user + "'"
    m = tbl_truth.search().where(" id = " + id_true).to_df()
    
    pandas_users.loc[index,'age_true'] = m['age'][0]
    pandas_users.loc[index,'gender_true'] = m['gender'][0]
    pandas_users.loc[index,'ethnicity_true'] = m['ethnicity'][0]

    n = tbl_generated.search().where(" id_true = " + id_true).to_df()
    pandas_users.loc[index,'age_generated'] = n['age'][0]
    pandas_users.loc[index,'gender_generated'] = n['gender'][0]
    pandas_users.loc[index,'ethnicity_generated'] = n['ethnicity'][0]


    print("real: " + m['face_path'][0])
    print("generated: " + n['generated_face_path'][0])

    
    im_true = cv2.imread(m['face_path'][0])
    im_generated = cv2.imread(n['generated_face_path'][0])
    psnr.append(cv2.PSNR(im_true, im_generated))
    #print(im_true.shape)
    ssim.append(calc_ssim(im_true,im_generated,channel_axis=2))

    mse.append(mean_squared_error(im_true,im_generated))
    

    

In [ ]:
f.write('Average PSNR is: ' + str(np.array(psnr).mean()) + "\n")
f.write('Average SSIM is: ' + str(np.array(ssim).mean()) + "\n")
f.write('Average  MSE is: ' + str(np.array(mse).mean()) + "\n")

In [ ]:
from math import sqrt
age_true = pandas_users['age_true'].to_numpy()
age_generated = pandas_users['age_generated'].to_numpy()
age_rmse = sqrt(((age_true - age_generated)**2).mean(axis = 0))
f.write('rmse of age: ' + str(age_rmse) + "\n")

In [ ]:
display_labels = pandas_users.gender_true.drop_duplicates(keep='first', inplace=False).to_numpy()
cm = confusion_matrix(pandas_users.gender_true, pandas_users.gender_generated,labels=display_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=display_labels,)
disp.plot(cmap="Blues")
disp.figure_.savefig(evaluation_results_folder + "/gender_cm.png",dpi=300)

In [ ]:
display_labels = pandas_users.ethnicity_true.drop_duplicates(keep='first', inplace=False).to_numpy()
cm = confusion_matrix(pandas_users.ethnicity_true, pandas_users.ethnicity_generated, labels=display_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=display_labels)
disp.plot(cmap="Blues")
disp.figure_.savefig(evaluation_results_folder + "/ethnicity_cm.png",dpi=300)

In [ ]:
pandas_users

In [ ]:
f.close()

In [ ]:
#https://pypi.org/project/pytorch-fid-wrapper/
#https://github.com/mseitzer/pytorch-fid

import  pytorch_fid_wrapper as pfw
import torchvision.datasets as datasets

data = datasets.ImageFolder(datasetPathGeneratedFaces)



